In [36]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate();
using RobotZoo: YakPlane
using RobotDynamics
using ForwardDiff
using StaticArrays
using LinearAlgebra
using Rotations
using Printf
using Test
const RD = RobotDynamics
include("utils.jl")
include("airplane.jl")

 Activating environment at `~/Classes/OptimalControl/hw4_solutions/Project.toml`


set_mesh! (generic function with 1 method)

# Q1: iLQR with Quaternions
In this problem you'll adapt the iLQR algorithm you wrote in the last homework to work with 3D rotations, using the methods we talked about in class. We'll also see one of the drawbacks of DDP-based methods like iLQR: the need for good initialization. 

In this problem we'll be designing a aerobatic maneuver for a small-scale model airplane. The dynamics you'll be using use data extracted from real-world wind-tunnel testing. Since the dynamics are fairly complex, we'll skip describing them in detail. Instead we'll simply describe the state and control inputs:

$$ x = \begin{bmatrix}
    p \\ q \\ v \\ \omega
\end{bmatrix}, \;\; 
u = \begin{bmatrix}  
    u_\text{thr} \\ u_\text{ail} \\ u_\text{elev} \\ u_\text{rud}
\end{bmatrix}
$$
where $p \in \mathbb{R}^3$ is the position of the center of mass in the world frame, $q \in \mathbb{S}^3$ is the unit quaternion mapping vectors in the body frame to the world frame, $v \in \mathbb{R}^3$ is the velocity in the world frame, and $\omega \in \mathbb{R}^3$ is the angular velocity in the body frame. The controls are all scalar values ranging from 0-255 for the thrust, aileron, elevator, and rudder control surfaces.

If you want to look at the system dynamics, check out [this file](https://github.com/RoboticExplorationLab/RobotZoo.jl/blob/master/src/yak_plane.jl).


## Part (a): Finding the trim conditions
To get a good answer out of shooting methods like iLQR that rely on a dynamically feasible initial guess, it's often a good idea to find some sort of "stable" initial trajectory. For this problem, we'll simply find the "trim" conditions for level flight, or the controls need to maintain level flight with zero acceleration. In principle, this is going to be very similar to the problem in the HW1 where we found the equilibrium state for the quadruped, except that here we're only constraining the accelerations (not velocities) to be zero.

To find the trim conditions, we'll solve the following optimization problem using Gauss-Newton:
$$
\begin{align}
& \text{minimize}_{u} && \frac{1}{2} (u - u_\text{guess})^T (u - u_\text{guess}) \\
& \text{subject to} && f_\text{accel}(x_\text{trim},u) = 0
\end{align}
$$
where $u_\text{guess}$ in an initial guess for the controls, $x_\text{trim}$ is the state vector (both position and velocity) we want to design our trim controls for. In our case, this will be level flight with a forward velocity along the x-axis of 5 m/s. $f_\text{accel}$ is a function that returns the linear and angular accelerations. You can just pull these out of a call to the continuous-time dynamics function.

In [2]:
# TASK: implement get_trim to find the trim controls using Newton's method
"""
    get_trim(model::YakPlane, x_trim, u_guess)

Calculate the trim controls for the YakPlane model, given the position and velocity specified by `x_trim`.
Find a vector of 4 controls close to those in `u_guess` that minimize the accelerations on the plane.
"""
function get_trim(model::YakPlane, x_trim, u_guess;
        verbose = false,
        tol = 1e-4,
        iters = 100,
    )

    # TODO: Use Newton's method to find the trim controls
    #  TIP: Look at your solution the quadruped problem on HW1
    #  TIP: You should have 4 decision variables and 6 constraints
    #  TIP: You don't need to do anything special due to the rotation, just do normal (Gauss) Newton
    
    utrim = copy(u_guess)
    
    # SOLUTION
    a = zeros(3)       # linear acceleration
    α = zeros(3)       # angular acceleration
    n,m = size(model)

    iu = SVector{m}(1:m)           # control indices
    ic = SVector{6}(m .+ (1:6))    # constraint indices
    ia = SVector{6}(1:6) .+ 7      # acceleration indices

    # Jacobian of the constraint
    ∇f(u) = ForwardDiff.jacobian(u_->dynamics(model, x_trim, u_), u)[ia,:]

    # Residual function
    r(z) = [z[iu] - u_guess + ∇f(z[iu])'z[ic]; dynamics(model, x_trim, z[iu])[ia] ]

    # Jacobian of the the residual function
    ∇r(z) = begin
        u,λ = z[iu], z[ic] 
        B = ∇f(u)
        [I B'; B -I(6)*1e-6]
    end

    # Initial guess
    λ = @SVector zeros(6)
    z = [u_guess; λ]

    # Newton solve
    for i = 1:iters
        # Check convergence
        res = r(z)
        verbose && println(norm(res))
        if norm(res) < tol 
            verbose && println("converged in $i iterations")
            break
        end

        # Compute Newton step
        R = ∇r(z) 
        dz = -(R \ res)
        z += dz
    end
    utrim = z[iu]
    # END SOLUTION

    # Return the trim controls
    return utrim
end

get_trim (generic function with 1 method)

In [59]:
# Test that the method find zero acceleration
begin  
    model = YakPlane(UnitQuaternion)
    x0 = SA[
        0,0,0, 
        -6.114599630220157e-8, 0.997155939027903, 0.0, 0.07536599539167085,
        5,0,0,
        0,0,0
    ]
    u_guess = fill(124,4)
    utrim = get_trim(model, x0, u_guess)

    # Make sure the accelerations are small
    @test norm(dynamics(model, x0, utrim)[8:13]) < 1e-3
end # wrap in begin/end to avoid polluting the global state with variables

Test Passed

## Part (b): Computing the Error State
As we learned in class, we will compute the backward pass of iLQR on the error state, which for a single rigid body like the airplane will be an element of $\mathbb{R}^{12}$. We need a method for converting between our state vector in $\mathbb{R}^{9} \times \mathbb{S}^3$ into an error state. 

Before computing the error state, we need a few methods to work with quaternions. We'll need a way to multiply two quaternions. The easiest way to do this is by defining the linear operator $L(q_1) q_2$. Define the `lmult` method below.

In [61]:
# TASK: 

"""
    lmult(q)

Return the 4x4 matrix `L` such that `L(q1)*q2` is equivalent to the quaternion multiplication `q1*q2`,
where `q1` and `q2` follow the Hamilton convention (i.e. q[1] is the scalar part and q[2:4] is the vector part).
"""
function lmult(q)
    # TODO: calculate L(q)
    
    # SOLUTION
    SA[
        q[1] -q[2] -q[3] -q[4];
        q[2]  q[1] -q[4]  q[3];
        q[3]  q[4]  q[1] -q[2];
        q[4] -q[3]  q[2]  q[1];
    ]
    # END SOLUTION
end

# Generate two random unit quaternion
begin
    q1 = normalize(@SVector randn(4))
    q2 = normalize(@SVector randn(4))

    # A couple simple tests (feel free to use Rotations.jl to check your answers)
    @test UnitQuaternion(lmult(q2)*q1) ≈ UnitQuaternion(q2)*UnitQuaternion(q1)
    @test lmult(q1)'q1 ≈ [1,0,0,0];
end

Test Passed

There are many ways to represent a three-parameter error quaternion, but we'll use the *Cayley Map*, which we've found to work best and is very computationally efficient. This is equivalent to converting the quaternion difference to a Rodrigues Parameter:
$$ g = \frac{1}{q_s} \begin{bmatrix} q_x \\ q_y \\ q_z \end{bmatrix} $$

Using the Cayley map, compute the error state, given two states $x$ and $x_0$. This is equivalent to $x - x_0$ for vector (Euclidean) states. Remember that the difference between two quaternions is just $q_0^\dagger q$ which is the same as $L(q_0^\dagger) q = L(q_0)^T q$.

In [5]:
# TODO: Implement the state_error function
"""
    state_parts(model::YakPlane)
Give a tuple of indices, `ip,iq,iv,iw` for extracting pieces of the state vector:
* `x[ip]` returns the 3 positions in the world frame
* `x[iq]` returns the 4 elements of the unit quaternion
* `x[iv]` returns the 3 linear velocities in the world frame
* `x[iw]` returns the 3 angular velocities in the body frame
"""
function state_parts(model::YakPlane)
    ip = SA[1,2,3]
    iq = SA[4,5,6,7]
    iv = SA[8,9,10]
    iw = SA[11,12,13]
    return ip, iq, iv, iw
end


function state_error(model::YakPlane, x, x0)
    # TODO: Compute the error state between x and x0
#     dx = zeros(length(x)-1)
    
    # SOLUTION
    ip,iq,iv,iw = state_parts(model)
    q  = UnitQuaternion(x[iq])
    q0 = UnitQuaternion(x0[iq])
    dq = cayleymap(lmult(x0[iq])'x[iq]) 
    dx = [x[ip] - x0[ip]; dq; x[iv] - x0[iv]; x[iw] - x0[iw]]
    # END SOLUTION
    
    return dx
end
# SOLUTION
cayleymap(q) = SA[q[2],q[3],q[4]] / q[1]
# END SOLUTION

state_error (generic function with 1 method)

Now that we have a function to compute the error state, we're also going to need it's Jacobian (no surprise there). Remember that the *attitude Jacobian* is the Jacobian of the quaternion difference, and is given as:
$$
G(q) = \begin{bmatrix}
    -q_v^T \\ q_s I_3 + \hat{q}_v
\end{bmatrix}
$$
where $q_s \in \mathbb{R}$ and $q_v \in \mathbb{R}^{3}$ are the scalar and vector part of the quaternion, respectively, and $\hat{x}$ is the skew-symmetric matrix operator:
$$
\hat{x} = \begin{bmatrix}
 0   & -x_3 & x_2 \\
 x_3 &  0   &-x_1 \\
-x_2 &  x_1 & 0   \\
\end{bmatrix}
$$

In [64]:
# TODO: Implement the following methods
#       1. attitude_jacobian
#       2. state_error_jacobian

"""
    attitude_jacobian(q)

Return the 4x3 attitude Jacobian for the unit quaternion q following the Hamilton convention.
The attitude Jacobian is the Jacobian of `q*ϕ(dq)` with respect to `dq`,
where `ϕ` transforms the infinitessimal rotation `dq` to a unit quaternion.
"""
function attitude_jacobian(q)
#     return @SMatrix zeros(4,3)
    # SOLUTION
    SA[
        -q[2] -q[3] -q[4];
         q[1] -q[4]  q[3];
         q[4]  q[1] -q[2];
        -q[3]  q[2]  q[1];
    ]
    # END SOLUTION
end

"""
    state_error_jacobian(model, x)

Return the Jacobian of the `state_error` function, with respect to the state `x`.
"""
function state_error_jacobian(model, x)
#     return @SMatrix zeros(length(x), length(x)-1)
    
    # SOLUTION
    iq = state_parts(model)[2]
    q = x[iq] 
    G = attitude_jacobian(q)
    SA[
        1 0 0 0 0 0 0 0 0 0 0 0;
        0 1 0 0 0 0 0 0 0 0 0 0;
        0 0 1 0 0 0 0 0 0 0 0 0;
        0 0 0 G[1] G[5] G[9]  0 0 0 0 0 0;
        0 0 0 G[2] G[6] G[10] 0 0 0 0 0 0;
        0 0 0 G[3] G[7] G[11] 0 0 0 0 0 0;
        0 0 0 G[4] G[8] G[12] 0 0 0 0 0 0;
        0 0 0 0 0 0 1 0 0 0 0 0;
        0 0 0 0 0 0 0 1 0 0 0 0;
        0 0 0 0 0 0 0 0 1 0 0 0;
        0 0 0 0 0 0 0 0 0 1 0 0;
        0 0 0 0 0 0 0 0 0 0 1 0;
        0 0 0 0 0 0 0 0 0 0 0 1;
    ]
    # END SOLUTION
end

state_error_jacobian

In [66]:
# Test the attitude and state error Jacobians
begin
    @test attitude_jacobian(q1) ≈ Rotations.∇differential(UnitQuaternion(q1))
    G = state_error_jacobian(model, x0)
    @test size(G) == (state_dim(model), state_diff_size(model))
    @test G[1:3,1:3] == I(3)
    @test G[8:end, 7:end] == I(6)
    @test G[4:7,4:6] == attitude_jacobian(x0[4:7])
end

Test Passed

## Part (c): A Better Cost Function
In our trajectory optimization problem, we're going to want to penalize distance from a reference trajectory or a goal, so we need a cost function that does this efficiently and correctly for our state vector, which includes quaternions. While we could use the error we computed above, this is very nonlinear and can impede convergence. Instead, we use the simple *geodesic distance* between two quaternions:
$$ \text{dist}_\text{geo}(q_1,q_2) = \min(1 \pm q_1^T q_2) $$

For our trajectory optimization we're going to use a cost function of the following form:
$$ \frac{1}{2}\left( (p-p_d)^T Q_p (p-p_d) + (v-v_d)^T Q_v (v-v_d) + (\omega - \omega_d)^T Q_\omega (\omega - \omega_d) \right)  + w \cdot \text{dist}_\text{geo}(q, q_d)$$
Use this to flush out the cost function below, and it's derivatives.

In [73]:
# TASK: Complete the following methods
#       1. stagecost
#       2. termcost
#       3. gradient
#       4. hessian

"""
    RigidBodyCost{m,T}

A cost function that penalizes distance of a rigid body state specified by its 
position `p`, attitude `q` (a unit quaternion), linear velocity `v` and angular velocity `ω`.

The vector state use a standard quadratic penalty, and the attitude uses the geodesic distance, weighted by a scalar `w`.
Uses a quadratic penalty on the distance of the `m`-dimensional control vector from a reference control `u_ref`.
"""
struct RigidBodyCost{m,T}
    Qp::Diagonal{T,SVector{3,T}}
    w::T
    Qv::Diagonal{T,SVector{3,T}}
    Qω::Diagonal{T,SVector{3,T}}
    R::Diagonal{T,SVector{m,T}}
    p_ref::SVector{3,T}
    q_ref::SVector{4,T}
    v_ref::SVector{3,T}
    ω_ref::SVector{3,T}
    u_ref::SVector{m,T}
end

"""
    stagecost(cost::RigidBodyCost, x, u)

Calculate the cost for a single knot point for the `RigidBodyCost`. 
TIP: Call `termcost` and add the control cost, to avoid repeated code
"""
function stagecost(cost::RigidBodyCost, x, u)
    # TODO: Calculuate the stage cost
    J = termcost(cost, x)
    
    # SOLUTION
    du = u - cost.u_ref
    J += 0.5*du'cost.R*du
    # END SOLUTION
    
    return J
end

"""
    termcost(cost::RigidBodyCost, x)

Calculate the state-only terminal cost for a `RigidBodyCost`. 
Should use the geodesic distance to compute the error between the 
quaternion in `x` and `cost.q_ref`.
"""
function termcost(cost::RigidBodyCost, x)
    # TODO: Calculate the state-only cost
    J = 0
    
    # SOLUTION
    p = SA[x[1], x[2], x[3]] - cost.p_ref
    q = SA[x[4],x[5],x[6],x[7]]
    v = SA[x[8], x[9], x[10]] - cost.v_ref
    ω = SA[x[11], x[12], x[13]] - cost.ω_ref

    J = 0.5*(p'cost.Qp*p + v'cost.Qv*v + ω'cost.Qω*ω)
    dq = cost.q_ref'q
    J += cost.w * min(1+dq, 1-dq)
    # END SOLUTION
    
    return J
end

"""
    gradient(cost::RigidBodyCost, x, u)

Calculate the gradient of the cost function with respect to
the states `x` and controls `u`. Should return both as a tuple.
"""
function gradient(cost::RigidBodyCost, x, u)
    # TODO: Calculate the gradients with respect to both x and u
#     grad_x = zero(x)
#     grad_u = zero(u)
    
    # SOLUTION
    p = SA[x[1], x[2], x[3]] - cost.p_ref
    q = SA[x[4],x[5],x[6],x[7]]
    v = SA[x[8], x[9], x[10]] - cost.v_ref
    ω = SA[x[11], x[12], x[13]] - cost.ω_ref

    dq = cost.q_ref'q
    s = dq < 0 ? 1 : -1
    grad = [
        cost.Qp*p;
        cost.w*cost.q_ref*s;
        cost.Qv*v;
        cost.Qω*ω;
    ]
    grad_x = grad
    grad_u = cost.R*(u- cost.u_ref)
    # END SOLUTION
    
    return grad_x, grad_u
end

"""
    hessian(cost::RigidBodyCost, x, u)

Calculate the Hessian of the cost function with respect to both the 
states `x` and controls `u` for a `RigidBodyCost`. 
Should return a tuple of both Hessians
"""
function hessian(cost::RigidBodyCost, x, u)
    # TODO:
    Q = @SMatrix zeros(length(x), length(x))
    R = @SMatrix zeros(length(u), length(u))
    
    # SOLUTION
    Q = Diagonal([diag(cost.Qp); (@SVector zeros(4)); diag(cost.Qv); diag(cost.Qω)])
    R = cost.R
    # END SOLUTION
    
    return Q, R
end

"""
    cost(obj, X, U)

Calculate the cost along an entire trajectory using a collection
of cost function types (stored in the iteratable collection `obj`).
"""
function cost(obj, X, U)
    J = zero(eltype(X[1]))
    for k = 1:length(U)
        J += stagecost(obj[k], X[k], U[k])
    end
    J += termcost(obj[end], X[end])
    return J
end

cost

In [74]:
# Test cost and derivatives
begin
    model = YakPlane(UnitQuaternion)
    x,u = rand(model)
    ip,iq,iv,iw = state_parts(model)
    costfun = RigidBodyCost(
        Diagonal(@SVector fill(0.1, 3)),
        10.0, 
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(1e-3, 4)),
        x[ip],
        x[iq],
        x[iv],
        x[iw],
        u
    )
    
    J1 = stagecost(costfun, x, u)
    xflip = [x[ip]; -x[iq]; x[iv]; x[iw]]
    @test stagecost(costfun, xflip, u) ≈ J1

    model = YakPlane(UnitQuaternion)
    x,u = rand(model)
    stagecost(costfun, x,u)
    @test ForwardDiff.gradient(x->termcost(costfun,x), x) ≈ gradient(costfun, x, u)[1]
    @test ForwardDiff.gradient(u->stagecost(costfun,x,u), u) ≈ gradient(costfun, x, u)[2]
    @test ForwardDiff.hessian(x->termcost(costfun,x), x) ≈ hessian(costfun, x, u)[1]
    @test ForwardDiff.hessian(u->stagecost(costfun,x,u), u) ≈ hessian(costfun, x, u)[2]
end

Test Passed

## Part (d): Adapt the iLQR Algorithm
Now that we have all the building blocks, modify the iLQR algorithm to work with the airplane dynamics. You'll need to make modifications in two places:
1. The backward pass: You'll need to "convert" the Jacobians to be on the error state, by multiplying by the error state Jacobian. You'll need to handle the gradients, Hessians (remember the extra term) and dynamics Jacobians all slightly differently.
2. The forward pass: when simulating the dynamics forward, you'll need to apply the control law on the error state.

In [43]:
"""
    iLQRProblem{n,m,L}

Describes a trajectory optimization problem with `n` states, `m` controls, and 
a model of type `L`. 

# Constructor
    Problem(model::L, obj::Vector{<:QuadraticCost{n,m}}, tf, x0, xf) where {n,m,L}

where `tf` is the final time, and `x0` is the initial state. 
"""
struct iLQRProblem{n,m,L,O}
    model::L
    obj::Vector{O}
    N::Int
    tf::Float64
    x0::MVector{n,Float64}
    times::Vector{Float64}
    function iLQRProblem(model::L, obj::Vector{O}, tf, x0) where {L,O}
        n,m = size(model)
        @assert length(x0) == n
        T = length(obj)
        times = range(0, tf, length=T)
        new{n,m,L,O}(model, obj, T, tf, x0, times)
    end
end
Base.size(prob::iLQRProblem{n,m}) where {n,m} = (n,m,prob.N)

In [75]:
# TASK: Modify the backward pass to calculate the control law on the error state
"""
    backwardpass!(prob, P, p, K, d, X, U)

Evaluate the iLQR backward pass at state and control trajectories `X` and `U`, 
storing the cost-to-go expansion in `P` and `p` and the gains in `K` and `d`.

Should return ΔJ, expected cost reduction.
"""
function backwardpass!(prob::iLQRProblem{n,m}, P, p, K, d, X, U; 
        β=1e-6
    ) where {n,m}
    N = prob.N
    obj = prob.obj
    ΔJ = 0.0
    failed = false
    
    ∇f = RobotDynamics.DynamicsJacobian(prob.model) 
    ∇jac = zeros(n+m,n+m) 
    iq = state_parts(prob.model)[2]
    Iq = Diagonal(SA[0,0,0,1,1,1, 0,0,0, 0,0,0])

    # Calculate gradient and Hessian at terminal state
    Q, = hessian(prob.obj[N], X[N], 0*U[1])
    q, = gradient(prob.obj[N], X[N], 0*U[1])
    
    # TODO: Calculate the terminal cost-to-go, accounting for the group structure of the 3D rotation
    p[N]
    P[N]
    
    # SOLUTION
    G2 = state_error_jacobian(prob.model, X[N])
    p[N] = G2'q
    b = q[iq]'X[N][iq]
    P[N] = G2'Q*G2 - Iq*(q[iq]'X[N][iq])
    # END SOLUTION
    
    #Backward Pass
    for k = (N-1):-1:1

        # Cost Expansion
        Q,R =  hessian(prob.obj[k], X[k], U[k])
        q,r = gradient(prob.obj[k], X[k], U[k])

        # Dynamics derivatives
        dt = prob.times[k+1] - prob.times[k]
        z = KnotPoint(SVector{n}(X[k]), SVector{m}(U[k]), dt, prob.times[k])
        discrete_jacobian!(RK4, ∇f, model, z)
        A = RobotDynamics.get_static_A(∇f)
        B = RobotDynamics.get_static_B(∇f)

        # TODO: Account for the group structure of 3D rotations using the attitude Jacobian (via state_error_jacobian)
        
        # SOLUTION
        G1 = state_error_jacobian(prob.model, X[k])
        q = G1'q
        Q = G1'Q*G1 - Iq*(q[iq]'X[k][iq])
        A = G2'A*G1
        B = G2'B
        G2 = G1     
        # END SOLUTION
    
        gx = q + A'*p[k+1]
        gu = r + B'*p[k+1]
    
        Gxx = Q + A'*P[k+1]*A
        Guu = R + B'*P[k+1]*B
        Gux = B'*P[k+1]*A
        
        # Regularization
        Guu_reg = Guu + B'*β*I*B
        Gux_reg = Gux 
        Guu_reg = SMatrix{m,m}(Guu) + β*Diagonal(@SVector ones(m))
        
        # Calculate Gains
        d[k] .= Guu_reg\gu
        K[k] .= Guu_reg\Gux_reg
    
        # Cost-to-go Recurrence
        p[k] .= gx - K[k]'*gu + K[k]'*Guu*d[k] - Gux'*d[k]
        P[k] .= Gxx + K[k]'*Guu*K[k] - Gux'*K[k] - K[k]'*Gux
        ΔJ += gu'*d[k]
   
    end
    return ΔJ
end

backwardpass!

In [76]:
# TASK: Modify the forward pass algorithm to apply the control policy on the error state
"""
    forwardpass!(prob, X, U, K, d, ΔJ, J)

Evaluate the iLQR forward pass at state and control trajectories `X` and `U`, using
the gains `K` and `d` to simulate the system forward. The new cost should be less than 
the current cost `J` together with the expected cost decrease `ΔJ`.

Should return the new cost `Jn` and the step length `α`.
"""
function forwardpass!(prob::iLQRProblem{n,m}, X, U, K, d, ΔJ, J,
        Xbar = deepcopy(X), Ubar = deepcopy(U);
        max_iters=10,
    ) where {n,m}
    N = prob.N
    model = prob.model

    Jn = J
    α = 0.0
    
    # SOLUTION
    # Line Search
    Xbar[1] = X[1]
    α = 1.0
    Jn = Inf
    for i = 1:max_iters
        
        # Forward Rollout
        for k = 1:(N-1)
            t = prob.times[k]
            dt = prob.times[k+1] - prob.times[k]
            
            # TODO: apply the local feedback control on the error state
            
            # SOLUTION
            dx = state_error(model, Xbar[k], X[k])
            Ubar[k] = U[k] - α*d[k] - K[k]*dx
            Xbar[k+1] = discrete_dynamics(RK4, model, Xbar[k], Ubar[k], t, dt) 
            # END SOLUTION
        end
        
        # Calculate the new cost
        Jn = cost(prob.obj, Xbar, Ubar)

        # Check Armijo condition
        if Jn <= J - 1e-2*α*ΔJ
            break
        else
            # Backtrack
            α *= 0.5  
        end
        if i == max_iters 
            α = 0
        end
    end
    
    # Accept direction
    for k = 1:N-1
        X[k] = Xbar[k]
        U[k] = Ubar[k]
    end
    X[N] = Xbar[N]
    
    return Jn, α
end

forwardpass!

In [77]:
"""
    solve_ilqr(prob, X, U; kwargs...)

Solve the trajectory optimization problem specified by `prob` using iterative LQR.
Returns the optimized state and control trajectories, as well as the local control gains,
`K` and `d`.

Should return the optimized state and control trajectories `X` and `U`, and the 
list of feedback gains `K` and cost-to-go hessians `P`.
"""
function solve_ilqr(prob::iLQRProblem{n,m}, X0, U0; 
        iters=100,     # max iterations
        ls_iters=10,   # max line search iterations
        reg_min=1e-6,  # minimum regularizatio for the backwardpass
        verbose=0,     # print verbosity
        eps=1e-5,      # termination tolerance
        eps_ddp=eps    # tolerance to switch to ddp
    ) where {n,m}
    t_start = time_ns()
    Nx,Nu,Nt = size(prob)

    # Initialization
    # NOTE: some of these sizes have changed!
    N = prob.N
    p = [zeros(n-1) for k = 1:N]              # ctg gradient
    P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
    d = [zeros(m) for k = 1:N-1]              # feedforward gains
    K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
    Xbar = [@SVector zeros(n) for k = 1:N]    # line search trajectory
    Ubar = [@SVector zeros(m) for k = 1:N-1]  # line search trajectory
    ΔJ = 0.0

    # Don't modify the trajectories that are passed in
    X = deepcopy(X0)
    U = deepcopy(U0)

    # Initial cost
    J = cost(prob.obj, X, U)
    
    # Initialize parameters
    Jn = Inf
    iter = 0
    tol = 1.0
    β = reg_min
    while tol > eps 
        iter += 1
        
        # Backward Pass
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=β)

        # Forward Pass
        Jn, α = forwardpass!(prob, X, U, K, d, ΔJ, J, Xbar, Ubar, max_iters=ls_iters)

        if α === zero(α) 
            β = max(β*10, 1.0)
            # β *= 10 
        # elseif α === one(α)
        #     β = reg_min
        else 
            β = max(β/2, reg_min)
        end

        # Update parameters
        tol = maximum(norm.(d, Inf))
        β = max(0.9*β, reg_min)

        # Output
        if verbose > 0
            @printf("Iter: %3d, Cost: % 6.2f → % 6.2f (% 7.2e), res: % .2e, β= %.2e, α = %.3f, ΔJ = %.3e\n",
                iter, J, Jn, J-Jn, tol, β, α, ΔJ
            )
        end
        J = Jn

        if iter >= iters
            @warn "Reached max iterations"
            break
        end

    end
    println("Total Time: ", (time_ns() - t_start)*1e-6, " ms")
    return X,U,K,P
end

solve_ilqr

In [78]:
function YakProblems(;
        N = 101,
        vecstate=false,
        scenario=:barrellroll, 
        heading=0.0,  # deg
        Qpos=1.0,
        kwargs...
    )
    model = YakPlane(UnitQuaternion)

    n,m = size(model)
    ip,iq,iv,iw = state_parts(model)

    # Discretization
    tf = 1.25
    if scenario == :fullloop 
        # Double the time, keeping the same discretization
        tf *= 2
        N = (N-1)*2 + 1
    end
    dt = tf/(N-1)

    # Forward Velocity 
    vel = 5.0

    if scenario ∈ (:halfloop, :fullloop) 
        ey = @SVector [0,1,0.]

        # Heading
        dq = expm(SA[0,0,1]*deg2rad(heading))

        # Initial state
        p0 = MRP(0.997156, 0., 0.075366) # initial orientation (level flight)
        x0 = RD.build_state(model, [-3,0,1.5], p0, [vel,0,0], [0,0,0])

        # Climb
        pm = expm(SA[1,0,0]*deg2rad(180))*expm(SA[0,1,0]*deg2rad(90))
        xm = RD.build_state(model, [0,0,3.], pm, pm * [vel,0,0.], [0,0,0])

        # Top of loop
        pf = MRP(0., -0.0366076, 0.) * dq # final orientation (upside down)
        xf = RD.build_state(model, dq*[3,0,6.], pf, pf * [vel,0,0.], [0,0,0])
        pf2 = RotZ(deg2rad(heading-180))

        # Dive
        xm2 = RD.build_state(model, [-3,3,4.], pm * RotY(pi), [0,0,-vel], [0,0,0])

        # Terminal state
        xf2 = RD.build_state(model, [0,4,1.5], p0, [vel,0,0], [0,0,0])

        t_flat = 5 / (xf[2] - xf[1])
        N_flat = Int(round(t_flat/dt))

        # Xref trajectory
        x̄0 = RBState(model, x0)
        x̄m = RBState(model, xm)
        x̄m2 = RBState(model, xm2)
        x̄f = RBState(model, xf)
        x̄f2 = RBState(model, xf2)
        Xref = map(1:N) do k
            t = (k-1)/(N-1)
            Nmid = N ÷ 4
            if scenario == :fullloop
                if k < Nmid
                    x1 = x̄0
                    x2 = x̄m
                    t = (k-1)/Nmid
                elseif k < 2Nmid
                    t = (k-Nmid)/Nmid
                    x1 = x̄m
                    x2 = x̄f 
                elseif k < 3Nmid
                    t = (k-2Nmid)/Nmid
                    x1 = x̄f
                    x2 = x̄m2
                else
                    t = (k-3Nmid)/Nmid
                    x1 = x̄m2
                    x2 = x̄f2
                end
            else
                if k < 2Nmid
                    t = (k-1)/2Nmid
                    x1 = x̄0
                    x2 = x̄m
                else
                    t = (k-2Nmid)/(2Nmid+1)
                    x1 = x̄m
                    x2 = x̄f
                end
            end
            RBState(
                x1.r + (x2.r - x1.r)*t,
                slerp(x1.q, x2.q, t),
                x1.v + (x2.v - x1.v)*t,
                SA[0,pi/1.25,0]
            )
        end
    else
        throw(ArgumentError("$scenario isn't a known scenario"))
    end

    # Get trim condition
    utrim = get_trim(model, x0, fill(124, 4))

    # Objective
    Qf_diag = RD.fill_state(model, 10, 500*0, 100, 100.)
    Q_diag = RD.fill_state(model, Qpos*0.1, 0.1*0, 0.1, 1.1)
    R = Diagonal(@SVector fill(1e-3,4))
    costs = map(1:N-1) do k
        RigidBodyCost(
            Diagonal(Q_diag[ip])*dt,
            10.0*dt, 
            Diagonal(Q_diag[iv])*dt,
            Diagonal(Q_diag[iw])*dt,
            R*dt,
            Xref[k][ip],
            Xref[k][iq],
            Xref[k][iv],
            Xref[k][iw],
            utrim
        )
    end
    costterm = RigidBodyCost(
            Diagonal(Qf_diag[ip]),
            200.0, 
            Diagonal(Qf_diag[iv]),
            Diagonal(Qf_diag[iw]),
            R*0,
            Xref[N][ip],
            Xref[N][iq],
            Xref[N][iv],
            Xref[N][iw],
            utrim
        )
    push!(costs, costterm)

    # Build problem
    prob = iLQRProblem(model, costs, tf, x0)

    return prob, Xref, utrim
end

YakProblems (generic function with 1 method)

## Part (e): Solve for half of the loop
It turns out the maneuver we're trying to execute is a little tricky to get with a simple trim condition initializer (remember that these problems have lots of local minima). To get the desired behavior, we're going to first solve a simpler problem that will give us a good initialization for the problem we actually care about. We're basically going to solve for half of the loop first, and then ask it to get the rest of the loop.

We've taken care of the reference trajectories for this problem, but we encourage you to look at the code and at the trajectories in the visualizer.

#### Visualizer

In [79]:
vis = Visualizer()
set_mesh!(vis, model)
render(vis)  # this may take a while, it's a detailed model

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8705
└ @ MeshCat /home/brian/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x86, 0xa6  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8705))

In [80]:
# Generate the problem, and visualize the reference trajectory
prob_half, Xref_half = YakProblems(costfun=:QuatLQR, scenario=:halfloop, heading=130)
visualize!(vis, prob_half.model, prob_half.tf, Xref_half)

In [84]:
# TASK: Generate the initial guess, using trim conditions
#  TIP: calculate the trim controls using get_trim
#  TIP: write a method to simulate the dynamics forward

U0 = [@SVector zeros(control_dim(prob_half.model)) for k = 1:prob_half.N-1]
X0 = [SVector(prob_half.x0) for k = 1:prob_half.N]      # remember this needs to be dynamically feasible!

# SOLUTION
function rollout(model, x0::StaticVector{n}, U, times) where n
    N = length(U0) + 1
    X = [@SVector zeros(n) for k = 1:N]
    X[1] = x0
    for k = 1:N-1
        dt = times[k+1] - times[k]
        X[k+1] = discrete_dynamics(RK4, model, X[k], U[k], times[k], dt)
    end
    return X
end
utrim = get_trim(model, prob_half.x0, fill(124,4))
U0 = [copy(utrim) for k = 1:prob_half.N-1]
X0 = rollout(prob_half.model, prob_half.x0, U0, prob_half.times)
# END SOLUTION

visualize!(vis, prob_half.model, prob_half.tf, X0)

In [86]:
# Solve for the first half of the trajectory
# TIP: It should take about 17 iterations
# TIP: Final cost should be about 6.2

Xhalf, Uhalf = solve_ilqr(prob_half, X0, U0, verbose=1, eps=1e-4, reg_min=1e-6)
visualize!(vis, prob_half.model, prob_half.tf, Xhalf)

Iter:   1, Cost:  4878.39 →  4621.95 ( 2.56e+02), res:  5.81e+03, β= 1.00e-06, α = 0.063, ΔJ = 9.578e+03
Iter:   2, Cost:  4621.95 →  4296.81 ( 3.25e+02), res:  7.39e+03, β= 1.00e-06, α = 0.031, ΔJ = 8.863e+03
Iter:   3, Cost:  4296.81 →  3742.09 ( 5.55e+02), res:  5.41e+03, β= 1.00e-06, α = 0.063, ΔJ = 8.372e+03
Iter:   4, Cost:  3742.09 →  2828.06 ( 9.14e+02), res:  6.71e+03, β= 1.00e-06, α = 0.125, ΔJ = 7.357e+03
Iter:   5, Cost:  2828.06 →  1666.31 ( 1.16e+03), res:  5.23e+03, β= 1.00e-06, α = 0.500, ΔJ = 5.603e+03
Iter:   6, Cost:  1666.31 →  249.73 ( 1.42e+03), res:  2.15e+03, β= 1.00e-06, α = 1.000, ΔJ = 3.260e+03
Iter:   7, Cost:  249.73 →  103.16 ( 1.47e+02), res:  9.18e+02, β= 1.00e-06, α = 0.500, ΔJ = 4.800e+02
Iter:   8, Cost:  103.16 →   8.20 ( 9.50e+01), res:  4.93e+02, β= 1.00e-06, α = 1.000, ΔJ = 1.907e+02
Iter:   9, Cost:   8.20 →   6.74 ( 1.47e+00), res:  1.33e+02, β= 1.00e-06, α = 0.500, ΔJ = 3.955e+00
Iter:  10, Cost:   6.74 →   6.22 ( 5.17e-01), res:  7.12e+01, β= 

## Part (f): Complete the Loop
Use the result of your previous solve to generate a good initial guess for the full maneuver. The new trajectory is twice as long as the previous one.

**TIP**: Reverse the control sequence for the second half of the trajectory.

In [87]:
# Generate the new problem and reference trajectory
prob, Xref2 = YakProblems(costfun=:QuatLQR, scenario=:fullloop, heading=130, Qpos=100)
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x86, 0xa6  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8705))

In [90]:
# Visualize the reference trajectory
visualize!(vis, prob.model, prob.tf, Xref2)

In [91]:
# TASK: Design a good initialization, using result for the previous problem
U0 = [copy(utrim) for k = 1:prob_half.N-1]
X0 = rollout(prob.model, prob.x0, U0, prob.times)

# SOLUTION
U0 = [copy(utrim) for k = 1:prob.N-1]
for k = 1:length(Uhalf)
    U0[k] = Uhalf[k]
end
U0[1+length(Uhalf):end] .= reverse(Uhalf)
X0 = rollout(prob.model, prob.x0, U0, prob.times)
# END SOLUTION

visualize!(vis, prob.model, prob.tf, X0)

In [94]:
# Solve for the full loop
# TIP: This will probably take between 100-200 iterations
# TIP: Final cost should be about 14
# TIP: the regularization parameter has already been tuned, but feel free to change it if needed
# NOTE: We've relaxed the optimality condition for this solve, to make it easier

Xfull, Ufull = solve_ilqr(prob, X0, U0, verbose=1, eps=1e-2, reg_min=1e-8, iters=500)
visualize!(vis, prob.model, prob.tf, Xfull)

Iter:   1, Cost:  9892.82 →  5236.26 ( 4.66e+03), res:  1.00e+04, β= 1.00e-08, α = 0.250, ΔJ = 1.973e+04
Iter:   2, Cost:  5236.26 →  3970.20 ( 1.27e+03), res:  7.22e+03, β= 1.00e-08, α = 0.125, ΔJ = 1.043e+04
Iter:   3, Cost:  3970.20 →  3482.81 ( 4.87e+02), res:  6.06e+03, β= 1.00e-08, α = 0.063, ΔJ = 7.896e+03
Iter:   4, Cost:  3482.81 →  2999.30 ( 4.84e+02), res:  5.53e+03, β= 1.00e-08, α = 0.063, ΔJ = 6.922e+03
Iter:   5, Cost:  2999.30 →  2636.90 ( 3.62e+02), res:  6.09e+03, β= 1.00e-08, α = 0.063, ΔJ = 5.953e+03
Iter:   6, Cost:  2636.90 →  2296.77 ( 3.40e+02), res:  5.59e+03, β= 1.00e-08, α = 0.063, ΔJ = 5.228e+03
Iter:   7, Cost:  2296.77 →  1997.60 ( 2.99e+02), res:  4.41e+03, β= 1.00e-08, α = 0.063, ΔJ = 4.549e+03
Iter:   8, Cost:  1997.60 →  1759.06 ( 2.39e+02), res:  4.73e+03, β= 1.00e-08, α = 0.063, ΔJ = 3.948e+03
Iter:   9, Cost:  1759.06 →  1543.15 ( 2.16e+02), res:  4.36e+03, β= 1.00e-08, α = 0.063, ΔJ = 3.471e+03
Iter:  10, Cost:  1543.15 →  1285.20 ( 2.58e+02), res: 